# PROGETTO MACHINE LEARNING - LIFESTYLE

## INTRODUZIONE

In questo progetto si utilizzerà il toy database [diabetes](https://scikit-learn.org/stable/datasets/toy_dataset.html#diabetes-dataset) per predire il livello di progessione della malattia dopo un anno, a partire da 10 variabili misurate sui pazienti

## CARICAMENTO DATASET

Sono state importate le varie librerie python che potrebbero essere utilizzate nel progetto

In [1]:
from sklearn.datasets import load_diabetes
from sklearn import svm
from sklearn.model_selection import train_test_split, GridSearchCV, learning_curve, validation_curve
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.ensemble import BaggingRegressor

Si fa il download del database diabetes

In [3]:
diabetes = load_diabetes()

Analizzo la descrizione che offre sklearn per il toy dataset

In [4]:
print(diabetes.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, total serum cholesterol
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, total cholesterol / HDL
      - s5      ltg, possibly log of serum triglycerides level
      - s6      glu, blood sugar level

Note: Each of these 1

Dalla descrizione è emerso che i dati sono stati normalizzati

Con questo comando, posso analizzare la grandezza del toy dataset (442) e il significato delle feature presenti nel toy dataset che sono 10 features più il target:
1. age: l'età
2. sex: sesso
3. bmi: indice di massa corporea
4. bp: pressione arteriosa media
5. s1: colestorolo totale
6. s2: lipoproteine a bassa densità
7. s3: lipoproteine ad alta densità
8. s4: colesterolo totale
9. s5: trigliceridi
10. s6: livello di zucchero nel sangue


utilizzo pandas per dare un primo sguardo ai dati presenti nel dataset

In [6]:
diabetes_df = pd.DataFrame(data=diabetes.data, columns=diabetes.feature_names)
diabetes_df['target'] = diabetes.target
diabetes_df.describe()

,age,sex,bmi,bp,s1,s2,s3,s4,s5,s6,target
count,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,4.420000e+02,442.000000
mean,-3.634285e-16,1.308343e-16,-8.045349e-16,1.281655e-16,-8.835316e-17,1.327024e-16,-4.574646e-16,3.777301e-16,-3.830854e-16,-3.412882e-16,152.133484
std,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,4.761905e-02,77.093005
min,-1.072256e-01,-4.464164e-02,-9.027530e-02,-1.123996e-01,-1.267807e-01,-1.156131e-01,-1.023071e-01,-7.639450e-02,-1.260974e-01,-1.377672e-01,25.000000
25%,-3.729927e-02,-4.464164e-02,-3.422907e-02,-3.665645e-02,-3.424784e-02,-3.035840e-02,-3.511716e-02,-3.949338e-02,-3.324879e-02,-3.317903e-02,87.000000
50%,5.383060e-03,-4.464164e-02,-7.283766e-03,-5.670611e-03,-4.320866e-03,-3.819065e-03,-6.584468e-03,-2.592262e-03,-1.947634e-03,-1.077698e-03,140.500000
75%,3.807591e-02,5.068012e-02,3.124802e-02,3.564384e-02,2.835801e-02,2.984439e-02,2.931150e-02,3.430886e-02,3.243323e-02,2.791705e-02,211.500000
max,1.107267e-01,5.068012e-02,1.705552e-01,1.320442e-01,1.539137e-01,1.987880e-01,1.811791e-01,1.852344e-01,1.335990e-01,1.356118e-01,346.000000


Si verifica la presenza di valori nulli nel dataset:

In [7]:
diabetes_df.isna().sum()

age       0
sex       0
bmi       0
bp        0
s1        0
s2        0
s3        0
s4        0
s5        0
s6        0
target    0
dtype: int64

In [8]:
diabetes_df.isnull().sum()

age       0
sex       0
bmi       0
bp        0
s1        0
s2        0
s3        0
s4        0
s5        0
s6        0
target    0
dtype: int64

Dall'analisi è emerso che in questo dataset non è presente nessun dato mancante.

## VALIDAZIONE

- cos'è il random forest
- spiegazione del random forest
- piccola spiegazione del ensemble method
- perchè si deve usare il random forest regressor

Per questo progetto è stato scelto di utilizzare il **Random Forest** come modello di apprendimento. Il Random Forest è un tipo di ensemble method basato sul Decision Tree. 

Il **Decision Tree** un algoritmo di machine learning che opera mediante la costruzione di un albero di decisione, dove le scelte su quale percorso seguire vengono fatte in base ai valori soglia delle features nel dataset. In pratica, il Decision Tree può essere visualizzato come una serie di 'if' innestati, il che lo rende comprensibile anche per coloro che hanno familiarità con concetti informatici. Uno dei vantaggi principali del Random Forest rispetto al decision tree è la riduzione dell'overfitting.

Perchè si chiama Random Forest?
Perchè è costituito da un insieme di alberi decisionali (Forest) ed è di tipo **Bagging** (Random), un tipo di ensemble method in cui avviene il campionamento casuale con sostituzione (nella statistica questa tecnica viene chiamata **bootstrap**, significa che i dati possono essere scelti più di una volta) dei dati di addestramento per creare dei sottoinsieme di dati diversi. Dopo il campionamento casuale, i modelli vengono addestrati in maniera indipendente ed effettuata la media o la maggioranza (in base se il un modello di regressione o classificazione) delle previsioni producendo una stima più precisa.







Per questo progetto è stato scelto di utilizzare come modello di apprendimento il **Random Forest**, è un ensemble method della decision tree. Il *Decision Tree* è di un albero di decisione e le decisioni su quale foglia proseguire vengono prese dai valori soglia presenti nelle features del dataset. Praticamente il decision tree sono come tanti if innestati (nel caso leggesse una persona che bazzica nell'ambito informatico). L'ensemble method è un ottimizzazione di un modello di apprendimento. Nel Random Forest vengono creati tanti decision tree con diverse valori di soglia 